In [2]:
import kivy
from kivy.app import App 
from kivy.uix.label import Label
from kivy.uix.gridlayout import GridLayout
from kivy.uix.textinput import TextInput
from kivy.uix.button import Button
from kivy.uix.widget import Widget
from kivy.properties import ObjectProperty
from kivy.uix.floatlayout import FloatLayout
from kivy.graphics import Rectangle
from kivy.graphics import Color
from kivy.graphics import Line


class Touch(Widget):
    
    def __init__(self, **kwargs):
        super(Touch, self).__init__(**kwargs)

        with self.canvas:
            Color(1,0,0,.5, mode = 'rgba')
            self.rect = Rectangle(pos=(0,0), size = (50, 50))


    #btn = ObjectProperty(None)

    def on_touch_down (self, touch):
        self.rect.pos = touch.pos
        print("Mouse Down", touch)
        #self.btn.opacity = 0.5  

    def on_touch_move (self, touch):
        self.rect.pos = touch.pos
        print("Mouse Move", touch)

    def on_touch_up (self, touch):
        print("Mouse Up", touch)
        #self.btn.opacity = 1


class MyApp (App):
    def build (self):
        #return FloatLayout()
        return Touch()
#Run Program 

if __name__ == "__main__":
    MyApp().run()





Mouse Down <MouseMotionEvent spos=(0.03504380475594493, 0.04006677796327207) pos=(28.0, 23.99999999999997)>
Mouse Move <MouseMotionEvent spos=(0.03629536921151439, 0.04340567612687818) pos=(28.999999999999996, 26.00000000000003)>
Mouse Move <MouseMotionEvent spos=(0.03879849812265332, 0.048414023372287174) pos=(31.0, 29.000000000000018)>
Mouse Move <MouseMotionEvent spos=(0.04005006257822278, 0.05175292153589317) pos=(32.0, 31.00000000000001)>
Mouse Move <MouseMotionEvent spos=(0.04505632040050062, 0.06343906510851416) pos=(36.0, 37.99999999999998)>
Mouse Move <MouseMotionEvent spos=(0.04630788485607009, 0.06844741235392315) pos=(37.0, 40.99999999999997)>
Mouse Move <MouseMotionEvent spos=(0.04881101376720901, 0.07178631051752926) pos=(39.0, 43.00000000000003)>
Mouse Move <MouseMotionEvent spos=(0.056320400500625784, 0.09015025041736224) pos=(45.0, 53.999999999999986)>
Mouse Move <MouseMotionEvent spos=(0.06508135168961202, 0.10183639398998334) pos=(52.0, 61.00000000000002)>
Mouse Move